In [1]:
import plotly.express as px
import pandas as pd
from geopy.distance import geodesic
import requests
from concurrent.futures import ThreadPoolExecutor

from post_roe.query import BucketQuery as bq, BUCKET_BASE
from post_roe.distance import Distance
from post_roe.pipeline import trigger_pipeline
px.set_mapbox_access_token("pk.eyJ1IjoicGFycXVhciIsImEiOiJja3lpcXMycGUxbmF5MnBzZXVzMHBzaXl4In0.jz0tx-HTJWym8jWPa8lqiA")
# trigger_pipeline("gmaps")



In [ ]:


# import json
# from glob import glob

# paths = glob("/Users/parker/Development/post-roe/data/clinics/*")
# def _open(path):
#     with open(path) as f: 
#         data = json.load(f)
#     data['zip3'] = path.split("/")[-1].split('.')[0]
#     return data

# places = pd.DataFrame([_open(p) for p in paths])

# clinics = pd.json_normalize(places['results'].explode())
# clinics = clinics.drop_duplicates(subset=['place_id'])
# clinics['lat'] = clinics['geometry.location.lat']
# clinics['lon'] = clinics['geometry.location.lng']
# def _get_zip5(formatted_address):
#     try: 
#         return formatted_address.split(",")[2].strip().split(" ")[-1]
#     except Exception as e:
#         print(e, formatted_address)


# clinics['zip5'] = clinics['formatted_address'].dropna().apply(_get_zip5)
# gmaps_clinics = clinics[['lat','lon','zip5']].dropna().reset_index(drop=True)
# gmaps_clinics['lat_lon'] = gmaps_clinics.apply(lambda x: [x['lat'],x['lon']],axis=1)
# # gmaps_clinics['lat'][0]
# gmaps_clinics.to_feather("gs://ohdo-post-roe-dev/data/clinics_gmaps.feather")

In [34]:
clinics_gmaps = bq.clinics_query("gmaps")
clinics_main = bq.clinics_query("main_515")


In [53]:
origins = bq.census_zip3_query() 
zip3_origins = origins[['zip3','lat_lon']].to_dict(orient="records")
zip3_origins[0]

{'zip3': '301**', 'lat_lon': (34.3157908, -85.2951255)}

In [61]:
def process_async(request):
    """  
        Takes about 5 min at k=5, clinics = 1k
    """
    MAX_WORKERS = 50

    # request_json = request.get_json()
    # case = request_json.get("case")
    case = 'gmaps'

    def _invoke_get_drive_time(origin: dict) -> dict:
        API_BASE = "https://us-central1-ohdo-post-roe-359822.cloudfunctions.net"
        endpoint = f"{API_BASE}/post-roe-sls-dev-get-drive-time"
        params = {
            "case": case,
            "k": 5,
            "origin": {
                "lat_lon": list(origin['lat_lon']),
                "zip3": origin['zip3'],
            } 
        }
        resp = requests.post(endpoint, json=params)
        if resp.status_code != 200:
            print(resp.status_code, resp.text)
        else:
            return resp.json()
    
    # loads 8e2 zip3 origins
    origins = bq.census_zip3_query() 
    zip3_origins = origins[['zip3','lat_lon']].to_dict(orient="records")
    
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as exec:
        futures = exec.map(_invoke_get_drive_time, zip3_origins)
    
    drive_times = list(futures)
    # df = pd.DataFrame(list(futures))
    # bq.to_feather(df, f"drive_time_{case}") # this is brittle. errr
    resp = {
        "case": case,
        "drive_times": drive_times
    }
    return resp

# case = process_async("")
# case

In [26]:
origins = bq.census_zip3_query()
drive_time = pd.DataFrame([dt for dt in case['drive_times'] if dt is not None ])
df = origins.merge(drive_time, how="left").drop_duplicates(subset=['zip3']).drop(columns=["lat_lon"])
df = df.merge(bq.states_query(case=case)[['state','state_status']])
df['adi_decile'] = df['adi_median'].apply(lambda x: int(x/10))
df['_drive_time'] = df['drive_time'].dropna().apply(lambda x: int(x/60))
df

,state,zip3,lat,lon,population,adi_median,case,drive_time,state_status,adi_decile,_drive_time
0,AL,301**,34.315791,-85.295125,39975,79.0,gmaps,86.8,at_risk,7,1.0
1,AL,319**,32.372789,-84.843361,16044,84.0,gmaps,84.0,at_risk,8,1.0
2,AL,350**,33.568065,-86.712692,442190,73.0,gmaps,148.4,at_risk,7,2.0
3,AL,351**,33.506417,-86.624839,324591,61.0,gmaps,139.4,at_risk,6,2.0
4,AL,352**,33.506077,-86.798758,491463,72.0,gmaps,150.0,at_risk,7,2.0
...,...,...,...,...,...,...,...,...,...,...,...
877,WY,824**,44.438020,-108.408179,52930,48.0,gmaps,331.6,at_risk,4,5.0
878,WY,825**,43.103802,-108.847958,38910,56.0,gmaps,312.0,at_risk,5,5.0
879,WY,826**,43.106535,-106.629017,91238,52.5,gmaps,248.4,at_risk,5,4.0
880,WY,828**,44.714826,-106.872537,36958,31.0,gmaps,292.0,at_risk,3,4.0


In [31]:
# gmaps_clinics.to_feather("gs://ohdo-post-roe-dev/data/clinics_gmaps.feather")

# df.to_feather("gs://ohdo-post-roe-dev/data/drive_time_gmaps.feather")

# px.box(times, x='state', y='_drive_time')

In [60]:
# case

def draw_case(case="gmaps"):
    origins['type'] = "origin"
    destinations = bq.clinics_query(case)
    destinations['type'] = "clinic"
    df = pd.concat([origins, destinations]).reset_index(drop=True)
    px.scatter_mapbox(
            df,
            lat="lat",
            lon="lon",
            size_max=15,
            height=600,
            zoom=3,
            color="type",
            color_discrete_sequence=["lightgrey", "magenta"],
            title="case: gmaps"
        ).show()

draw_case("gmaps")

In [69]:
from sklearn.preprocessing import MinMaxScaler

def draw_correlation(case = "base"):
    times = bq.drive_time_query(case)
    times['_adi_decile'] = pd.qcut(times['adi_median'], 10, labels=False)
    times['_adi_quartile'] = pd.qcut(times['adi_median'], 4, labels=False)
    times[['_population']] =  MinMaxScaler().fit_transform(times[['population']])
    times['_loss'] = times['_drive_time'] * times['_population']
    loss_by_adi = times.groupby(['adi_decile']).agg(total_loss=("_loss","sum")).reset_index()
    px.scatter(loss_by_adi, x='adi_decile', y='total_loss', title=f"case: {case}", trendline='ols').show()

draw_correlation("alt")

In [70]:
draw_correlation("gmaps")